In [30]:
# import libraries
from arcgis.gis import GIS

In [31]:
# connect to guest GIS, the ArcGIS Enterprise instance
guest_gis = GIS("https://pdoscriptdev.esri.com/portal", "admin", verify_cert=False)

# connect to host GIS, the new ArcGIS Online instance
host_gis = GIS("https://envisioning.maps.arcgis.com", "GBushongENV")

Enter password: ········
Enter password: ········


In [32]:
# groups: aec-enterprise-group aec-enterprise-group-2
# search for the sample group in guest gis
guest_group = guest_gis.groups.search("aec-enterprise-group")[0]
guest_group

# search for sample groups in host gis
host_group = host_gis.groups.search("TEST-1")[0]
host_group

<Group title:"TEST-1" owner:GBushongENV>

In [33]:
# create a collaboration
description='Data sharing sample between Enterprise and Online'
host_collab = host_gis.admin.collaborations.create(name='Enterprise + Online', description=description,
                                                  workspace_name='Enterprise sharing',
                                                  workspace_description=description,
                                                  portal_group_id=host_group.id,
                                                  host_contact_first_name='Grace',
                                                  host_contact_last_name='Bushong',
                                                  host_contact_email_address='gbushong@esri.com',
                                                  access_mode='sendAndReceive')
host_collab

{'id': 'eb00db3a48d642d891bac7b55c8ec5ea',
 'name': 'Enterprise + Online',
 'description': 'Data sharing sample between Enterprise and Online',
 'collaborationHostPortalId': '9F2Q47vO7UfnIjjV',
 'config': {'ownerUsername': 'GBushongENV'},
 'created': 1544660272000,
 'modified': 1544660272000}

In [34]:
# get the list of workspaces available as part of the collaboration
host_collab.workspaces

[{'id': '309cc37f04634bd188a7508651acdfab',
  'name': 'Enterprise sharing',
  'description': 'Data sharing sample between Enterprise and Online',
  'collaborationId': 'eb00db3a48d642d891bac7b55c8ec5ea',
  'config': {'maxItemSizeInMB': 1024,
   'maxReplicationPackageSizeInMB': 5120,
   'copyByRefIfCopyFail': True},
  'created': 1544660272000,
  'modified': 1544660272000,
  'lastSyncSuccess': -1,
  'participantGroupLinks': [{'portalId': '9F2Q47vO7UfnIjjV',
    'portalGroupId': 'ee213017ab39402fb933d01c8bbe2e87',
    'portalGroupName': 'TEST-1',
    'config': {'enableRealtimeSync': True, 'copyFeatureServiceData': False},
    'accessMode': 'sendAndReceive',
    'canSyncWorkspace': False}],
  'participantsWithoutGroupLinks': []}]

In [35]:
# compose the list of collaboration workspaces and the privileges for each
config = [{host_collab.workspaces[0]['id']:'sendAndReceive'}]

# invite the guest GIS and download the invitation file
invite_file = host_collab.invite_participant(config_json=config, expiration=24, 
                                             guest_portal_url="https://pdoscriptdev.esri.com/portal",
                                             save_path="/Users/grac9792/Documents/pdo-scripts")

# print the path to the invite file
invite_file

'/Users/grac9792/Documents/pdo-scripts/collaboration-pdoscriptdev.esri.com.invite'

## Accept the invitation at the participant GIS

In [36]:
guest_gis.admin.collaborations.accept_invitation(first_name='Grace', last_name='Bushong',
                                                email='gbushong@esri.com',
                                                invitation_file=invite_file)

{'success': True}

In [37]:
# get the list of collaborations on the guest GIS and pick teh one created earlier
guest_gis.admin.collaborations.list()

[{'id': 'eb00db3a48d642d891bac7b55c8ec5ea',
  'name': 'Enterprise + Online',
  'description': 'Data sharing sample between Enterprise and Online',
  'collaborationHostPortalId': '9F2Q47vO7UfnIjjV',
  'config': {'ownerUsername': 'admin'},
  'created': 1544660287952,
  'modified': 1544660287952}]

In [38]:
# in this case, there are two collaborations and the second one is the relevant one (id)
guest_collab = guest_gis.admin.collaborations.list()[0]
type(guest_collab)

arcgis.gis.admin._collaboration.Collaboration

# Export the response file

In [39]:
# now export a response file that can be sent back to the host GIS
response_file = guest_collab.export_invitation(out_folder = "/Users/grac9792/Documents/pdo-scripts/")
response_file

'/Users/grac9792/Documents/pdo-scripts/collaboration-pdoscriptdev.esri.com.response'

## Join a Group to the Collaboration in the guest GIS

In [40]:
# first get the list of workspaces in the guest collaboration
guest_collab.workspaces

[{'id': '309cc37f04634bd188a7508651acdfab',
  'name': 'Enterprise sharing',
  'description': 'Data sharing sample between Enterprise and Online',
  'collaborationId': 'eb00db3a48d642d891bac7b55c8ec5ea',
  'config': {'maxItemSizeInMB': 1024,
   'maxReplicationPackageSizeInMB': 5120,
   'copyByRefIfCopyFail': True},
  'created': 1544660288659,
  'modified': 1544660288659,
  'lastSyncSuccess': -1,
  'participantGroupLinks': [{'portalId': '9F2Q47vO7UfnIjjV',
    'accessMode': 'sendAndReceive',
    'config': {}}],
  'participantsWithoutGroupLinks': [{'portalId': '39dde654-828b-4a97-ad02-92e952a33ff9',
    'accessMode': 'sendAndReceive',
    'config': {}}]}]

In [41]:
guest_group

<Group title:"aec-enterprise-group" owner:admin>

In [42]:
guest_collab.add_group_to_workspace(portal_group=guest_group, workspace=guest_collab.workspaces[0])

{'success': True}

In [43]:
## Accept the collboration response at the host GIS
host_collab.import_invitation_response(response_file)

{'success': True}

In [46]:
# verify the collaboration
print("Collaborations on host GIS:")
for collab in host_gis.admin.collaborations.list():
    print(collab.name)
    
print("-----------------------------")
print("Collaborations on guest GIS:")
for collab in guest_gis.admin.collaborations.list():
    print(collab.name)

Collaborations on host GIS:
Enterprise + Online
collab_One Step Collab
-----------------------------
Collaborations on guest GIS:
Enterprise + Online
collab_One Step Collab


In [45]:
# one-step collaboration alternative
host_gis.admin.collaborations.collaborate_with(guest_gis=guest_gis, collaboration_name='One Step Collab',
                                               collaboration_description='Data sharing in one step')

True